# Yet another classical approach - this time based on Ziqi Yuan et al's "PhoGAD: Graph-based Anomaly Behavior Detection with Persistent Homology Optimization"-https://arxiv.org/pdf/2401.10547

Constructs graphs with multi-feature edges, calculates Betti number statistics, + trains model

## DATABASE PREP
Code for reading in concatenated csv (benign + attack), sorting it by timestamp (thus combining the data as before extraction), retaining only relevant features/columns, and creating new csv

In [ ]:
import dask.dataframe as dd          # large‑CSV engine
import pandas as pd
import numpy as np
import networkx as nx
from gudhi import SimplexTree
from ripser import ripser
from persim import plot_diagrams
import matplotlib.pyplot as plt
import seaborn as sns

# -------- parameters ----------
CSV_PATH   = "mergedBFXSS.csv"   # or bot.csv later
TIME_COL   = "timestamp"
SRC, DST   = "src_ip", "dst_ip"
LABEL = "label"
WEIGHT_COL = "total_payload_bytes"            # can change
WIN        = "5min"                           # resample size
VR_EPS     = 2000                             # distance/weight threshold
MAX_DIM    = 2                                # up to β2
N_JOBS     = 4                                # parallel PH cores
# --------------------------------
cols = [TIME_COL, SRC, DST, WEIGHT_COL, LABEL,"packets_count", "duration","bytes_rate","syn_flag_counts"]

# Read CSV with specified columns, using object dtype for payload size
ddf = dd.read_csv(
    CSV_PATH,
    usecols=cols,
    assume_missing=True,
    dtype={WEIGHT_COL: 'object'}
)

# Manually parse datetime
ddf[TIME_COL] = dd.to_datetime(ddf[TIME_COL], errors='coerce')

# Set timestamp as index with sorting to ensure global ordering
ddf = ddf.set_index(TIME_COL, sorted=False)

# Save as sorted CSV
ddf.to_csv("BFXSS_sorted.csv", index=True, single_file=True)

## 1. Load dataset 

In [1]:
import pandas as pd
import networkx as nx
import numpy as np

# Constants
WEIGHT_COL = "total_payload_bytes"
SRC, DST = "src_ip", "dst_ip"
VR_EPS = 2000

# Load sorted CSV
sorted_ddf = pd.read_csv("BFXSS_sorted.csv", parse_dates=["timestamp"])

# Floor timestamps
sorted_ddf["window_start"] = sorted_ddf["timestamp"].dt.floor("5min")

# # Define start and end times
# start_time = pd.Timestamp("2018-02-23 12:30:00")
# end_time = pd.Timestamp("2018-02-23 12:40:00")

# # Filter
# anomaly_window_df = sorted_ddf[(sorted_ddf["timestamp"] >= start_time) & (sorted_ddf["timestamp"] < end_time)]
# # We'll skip grouping by 'window_start' since we have a single window now
# window_df = anomaly_window_df

## 2. Construct graph

Builds a NetworkX graph from an edge list and node features.
    
    :param edge_list: List of (src, dst, edge_attr_dict)
    :param node_features: Dict mapping node ID to feature vector
    :return: A NetworkX graph with nodes and edges added

Example:
Node features: {'IP1': [1.2, 3.4], 'IP2': [0.7, 1.5]}
Edge list: [('IP1', 'IP2', {'payload_size': 1024, 'duration': 5})]


Load dataset

build_graph function

In [ ]:
import networkx as nx
import numpy as np

def build_graph(edge_list, node_features):
    """
    Builds a NetworkX graph from an edge list and node features.
    
    :param edge_list: List of (src, dst, edge_attr_dict)
    :param node_features: Dict mapping node ID to feature vector
    :return: A NetworkX graph with nodes and edges added
    """
    G = nx.Graph()
    for node_id, features in node_features.items():
        G.add_node(node_id, features=np.array(features))
    for src, dst, edge_attrs in edge_list:
        G.add_edge(src, dst, **edge_attrs)
    return G

calculate edges + make label dictionary (benign/attack) for model training

In [3]:
# Candidate edge feature columns (pick meaningful subset)
EDGE_FEATURE_CANDIDATES = [
    "total_payload_bytes",    # always keep
    "packets_count",          # basic packet count
    "duration",               # flow duration
    "syn_flag_counts"         # simple TCP control feature
]

# Keep only those that exist in the dataframe
EDGE_FEATURE_COLS = [c for c in EDGE_FEATURE_CANDIDATES if c in sorted_ddf.columns]

LABEL_MAPPING = {
    "Benign": 0,
    "Brute_Force_XSS": 1
}

# Graph construction
graphs = []

for window_start, window_df in sorted_ddf.groupby("window_start", sort=False):
    edge_list = []

    # Build edge list with rich feature vectors
    for _, row in window_df.iterrows():
        vol = row[WEIGHT_COL]
        if pd.isna(vol) or vol <= 0:
            continue

        edge_attrs = {feat: float(row[feat]) for feat in EDGE_FEATURE_COLS}
        edge_attrs["weight"] = min(float(vol), VR_EPS)

        edge_list.append((row[SRC], row[DST], edge_attrs))

    if not edge_list:
        continue

    # create label dictionary for model training
    label_dict = {}
    for _, row in window_df.iterrows():
        src = row["src_ip"]
        dst = row["dst_ip"]
        label_str = row["label"]
        label_num = LABEL_MAPPING.get(label_str, 0)  # Default to 0 if missing
        edge_key = (src, dst)
        if edge_key not in label_dict:
            label_dict[edge_key] = label_num
        else:
            label_dict[edge_key] = max(label_dict[edge_key], label_num)

    # Build temporary graph to compute node-level features
    G_temp = nx.Graph()
    G_temp.add_edges_from([(s, d, {"weight": a["weight"]}) for s, d, a in edge_list])

    node_features = {}
    for node in G_temp.nodes:
        degree = G_temp.degree(node)
        strength = sum(d["weight"] for _, _, d in G_temp.edges(node, data=True))
        node_features[node] = [degree, strength]

    # Build final graph
    G = build_graph(edge_list, node_features)
    graphs.append((window_start, G))


## Persistent Homology (PH) optimization

In [4]:
import gudhi as gd

def apply_persistent_homology_optimization(G, alpha=0.7):
    """
    Applies persistent homology optimization on edge attributes.
    
    :param G: NetworkX graph
    :param alpha: Weight for self-edge attribute
    :return: None (updates graph edge attributes in-place)
    """
    # Step 1: Collect edge features as points in feature space
    edge_points = []
    edge_mapping = []
    for u, v, data in G.edges(data=True):
        feat = np.array([data.get('payload_size', 0), data.get('duration', 0)])
        edge_points.append(feat)
        edge_mapping.append((u, v))
    
    # Step 2: Build Vietoris-Rips complex
    rips = gd.RipsComplex(points=edge_points, max_edge_length=2.0)
    simplex_tree = rips.create_simplex_tree(max_dimension=2)
    diag = simplex_tree.persistence()
    
    # Step 3: Identify persistent structures
    persistent_edges = set()
    for interval in diag:
        dim, (birth, death) = interval
        if dim == 1 and (death - birth) > 0.5:  # threshold can be adjusted
            # Assuming we treat all 1D holes as persistent structures
            persistent_edges.update(range(len(edge_points)))
    
    # Step 4: Update attributes
    for idx, (u, v) in enumerate(edge_mapping):
        current_feat = np.array([G[u][v].get('payload_size', 0), G[u][v].get('duration', 0)])
        if idx in persistent_edges:
            persistent_feats = [np.array(edge_points[j]) for j in persistent_edges]
            mean_feat = np.mean(persistent_feats, axis=0)
            updated_feat = alpha * current_feat + (1 - alpha) * mean_feat
            G[u][v]['payload_size'], G[u][v]['duration'] = updated_feat[0], updated_feat[1]


## Explicit edge embedding

In [5]:
def compute_edge_weight(u_feat, v_feat):
    num = np.dot(u_feat, v_feat)
    denom = np.linalg.norm(u_feat) * np.linalg.norm(v_feat) + 1e-8  # avoid division by zero
    return num / denom

def explicit_edge_embedding(G):
    """
    Computes explicit edge embeddings with neighbor aggregation.
    
    :param G: NetworkX graph
    :return: Dict mapping edge (u,v) to embedding vector
    """
    edge_embeddings = {}
    for u, v, data in G.edges(data=True):
        current_feat = np.array([data.get('payload_size', 0), data.get('duration', 0)])
        neighbor_feats = []
        
        # Collect neighbor edges
        neighbor_edges = set(G.edges(u)).union(set(G.edges(v)))
        neighbor_edges.discard((u, v))
        neighbor_edges.discard((v, u))
        
        for nu, nv in neighbor_edges:
            n_feat_u = G.nodes[nu]['features']
            n_feat_v = G.nodes[nv]['features']
            weight = compute_edge_weight(n_feat_u, n_feat_v)
            neighbor_feat = np.array([
                G[nu][nv].get('payload_size', 0),
                G[nu][nv].get('duration', 0)
            ])
            neighbor_feats.append(weight * neighbor_feat)
        
        # Aggregate neighbor features
        if neighbor_feats:
            agg_neighbors = np.mean(neighbor_feats, axis=0)
        else:
            agg_neighbors = np.zeros_like(current_feat)
        
        embedding = np.concatenate([current_feat, agg_neighbors])
        edge_embeddings[(u, v)] = embedding
    return edge_embeddings


## Anomaly detection module - PhoGAD reconstruction

In [6]:
import torch
from torch import nn
import torch.nn.functional as F

class PhoGADClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(PhoGADClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 2)  # binary classification

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

def focal_loss(pred, target, gamma=2.0):
    """
    Computes focal loss as in Eq. (12) from the paper.
    
    :param pred: predicted logits (before softmax)
    :param target: ground truth labels
    :param gamma: focusing parameter
    """
    logpt = -F.cross_entropy(pred, target, reduction='none')
    pt = torch.exp(logpt)
    loss = -((1 - pt) ** gamma) * logpt
    return loss.mean()

## Training loop

In [7]:
def train_model(model, edge_embeddings, edge_labels, epochs=10, lr=0.001):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.train()
    
    X = torch.tensor(edge_embeddings, dtype=torch.float32)
    y = torch.tensor(edge_labels, dtype=torch.long)
    
    for epoch in range(epochs):
        optimizer.zero_grad()
        output = model(X)
        loss = focal_loss(output, y)
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


# ! MAIN

In [8]:
# import glob

# X_files = sorted(glob.glob("edge_data/X_*.npy"))
# y_files = sorted(glob.glob("edge_data/y_*.npy"))

# X_all = np.vstack([np.load(f) for f in X_files])
# y_all = np.hstack([np.load(f) for f in y_files])

In [9]:
# 1. Build graph
G = build_graph(edge_list, node_features)

# 2. Apply persistent homology optimization
apply_persistent_homology_optimization(G)

# 3. Compute edge embeddings
edge_embeddings_dict = explicit_edge_embedding(G)
edge_embeddings = np.array(list(edge_embeddings_dict.values()))
edge_labels = []
for u, v in edge_embeddings_dict.keys():
    if (u, v) in label_dict:
        label = label_dict[(u, v)]
    elif (v, u) in label_dict:  # flip direction
        label = label_dict[(v, u)]
    else:
        label = 0  # default to normal if label is missing (optional)
    edge_labels.append(label)
edge_labels = np.array(edge_labels)


# 4. Initialize and train classifier
input_dim = edge_embeddings.shape[1]
model = PhoGADClassifier(input_dim, hidden_dim=16)
train_model(model, edge_embeddings, edge_labels, epochs=20)

Epoch 1, Loss: 1.5037
Epoch 2, Loss: 1.3879
Epoch 3, Loss: 1.2783
Epoch 4, Loss: 1.1748
Epoch 5, Loss: 1.0774
Epoch 6, Loss: 0.9863
Epoch 7, Loss: 0.9013
Epoch 8, Loss: 0.8225
Epoch 9, Loss: 0.7496
Epoch 10, Loss: 0.6821
Epoch 11, Loss: 0.6195
Epoch 12, Loss: 0.5613
Epoch 13, Loss: 0.5068
Epoch 14, Loss: 0.4554
Epoch 15, Loss: 0.4065
Epoch 16, Loss: 0.3597
Epoch 17, Loss: 0.3144
Epoch 18, Loss: 0.2705
Epoch 19, Loss: 0.2278
Epoch 20, Loss: 0.1868


## Evaluate on test set

In [10]:
import torch
from sklearn.metrics import classification_report

# Switch to evaluation mode
model.eval()

# Inference
with torch.no_grad():
    logits = model(torch.tensor(edge_embeddings, dtype=torch.float32))
    predictions = torch.argmax(logits, dim=1).numpy()

print(classification_report(
    edge_labels, predictions,
    labels=[0, 1],  # specify both classes explicitly
    target_names=["Benign", "Brute_Force_XSS"]
))
# print(classification_report(edge_labels, predictions, target_names=["Benign", "Brute_Force_XSS"]))

                 precision    recall  f1-score   support

         Benign       1.00      0.91      0.95        11
Brute_Force_XSS       0.00      0.00      0.00         0

       accuracy                           0.91        11
      macro avg       0.50      0.45      0.48        11
   weighted avg       1.00      0.91      0.95        11



/mnt/c/users/maryc/onedrive/desktop/Quantum Research 2025/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/mnt/c/users/maryc/onedrive/desktop/Quantum Research 2025/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/mnt/c/users/maryc/onedrive/desktop/Quantum Research 2025/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control th

In [11]:
print("Label distribution:", np.bincount(edge_labels))


Label distribution: [11]


In [12]:
# # 1. Build graph
# G = build_graph(edge_list, node_features)

# # 2. Apply persistent homology optimization
# apply_persistent_homology_optimization(G)

# # 3. Compute edge embeddings
# edge_embeddings_dict = explicit_edge_embedding(G)
# edge_embeddings = np.array(list(edge_embeddings_dict.values()))
# edge_labels = []
# for u, v in edge_embeddings_dict.keys():
#     if (u, v) in label_dict:
#         label = label_dict[(u, v)]
#     elif (v, u) in label_dict:  # flip direction
#         label = label_dict[(v, u)]
#     else:
#         label = 0  # default to normal if label is missing (optional)
#     edge_labels.append(label)
# edge_labels = np.array(edge_labels)


# # 4. Initialize and train classifier
# input_dim = edge_embeddings.shape[1]
# model = PhoGADClassifier(input_dim, hidden_dim=16)
# train_model(model, edge_embeddings, edge_labels, epochs=20)